# Synthetic data. Previous study.

https://github.com/albertovpd/viu_tfm-deep_vision_classification/tree/synthetic_data_study


In [1]:
# Google Drive stuff
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Mar 17 22:45:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    46W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# tf
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


- libs

In [4]:
%tensorflow_version 2.x
# batch ingestion of pics without pickle
from tensorflow.keras.preprocessing import image_dataset_from_directory

# nns
from tensorflow.keras.applications import ResNet50 

from tensorflow.keras import Model
from tensorflow.keras.models import load_model # Sequential
from tensorflow.keras import layers 

# optimization
from tensorflow.keras.optimizers import SGD #Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

# nn architectures, metrics, viz & reports => written in my_functions202202 file
import sys
sys.path.append("/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/src")
from my_functions202202 import generic_last_2layers, plotting_model, model_evaluation, classification_report_pic, confusion_matrix_report

import numpy as np
%matplotlib inline

# navigating through folder
import os

- paths

In [5]:
base_folder = "/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/"
test_folder = base_folder+"input/dataset_1test_5trainval_folders/test_ds/"
print(test_folder)
# regular folders for train set
reg_input = base_folder+"input/dataset_1test_5trainval_folders/train_val_ds/trainval_regular_partitions/"
src_folder =  base_folder+"src/"
output_folder = base_folder + "/output/"

/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision/input/dataset_1test_5trainval_folders/test_ds/


In [6]:
output_folder

'/content/drive/My Drive/2-Estudios/viu-master_ai/tfm-deep_vision//output/'

- functions

In [7]:
#my_functions202202.py

- common parameters

In [8]:
image_size = (128,128)
batch_size = 128
epochs = 250
opt = SGD(momentum=0.9) 

In [9]:
# available models
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(output_folder) if (isfile(join(output_folder, f)) & (".h5" in f) )]

for files in sorted(onlyfiles):
  print(files)

resnet50_NOdataAug_NOdropout.h5
resnet50_NOdataAug_dropoutFirst007.h5
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold1.h5
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold2.h5
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold3.h5
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold4.h5
resnet50_NOdataAug_dropoutFirst02.h5
resnet50_NOdataAug_dropoutMid02.h5
resnet50_dataAug_NOdropout.h5
resnet50_dataAug_dropoutFirst02.h5
resnet50_dataAug_dropoutMid02.h5
vgg19_NOdataAug_NOdropout_gelu.h5
vgg19_NOdataAug_dropoutFirst02.h5
vgg19_dataAug_dropoutFirst02.h5
vgg19_dataAug_dropoutFirst02_gelu.h5


In [58]:
# take the model for folder1
#base_model_resnet50 = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3), classes = 5) 
models_dict = {"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5": load_model(output_folder+"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5"),
               "resnet50_NOdataAug_dropoutFirst007_regKfolds_fold1.h5": load_model(output_folder+"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold1.h5"),
               "resnet50_NOdataAug_dropoutFirst007_regKfolds_fold2.h5": load_model(output_folder+"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold2.h5"),
               "resnet50_NOdataAug_dropoutFirst007_regKfolds_fold3.h5": load_model(output_folder+"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold3.h5"),
               "resnet50_NOdataAug_dropoutFirst007_regKfolds_fold4.h5": load_model(output_folder+"resnet50_NOdataAug_dropoutFirst007_regKfolds_fold4.h5")
              }
#nn = models_dict["resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5"]
#name = "resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5"[:-3]
#name

In [59]:
folders = os.listdir(reg_input)
folders

['fold2', 'fold4', 'fold0', 'fold3', 'fold1']

In [77]:
def inferences_target_list(model, data):
    '''
    returns 2 lists: inferences list, real labels
    '''
    # over train set fold1
    y_pred_float = model.predict(data)
    y_pred = np.argmax(y_pred_float, axis=1)

    # get real labels
    y_target = tf.concat([y for x, y in data], axis=0) 
    y_target
    print("lenght inferences and real labels: ", len(y_pred), len(y_target))
    return y_pred, y_target


def get_missclassified(y_pred, y_target):
  '''
  returns a list with the indexes of real labels that were missclassified
  '''
  missclassified = []
  for i, (pred, target) in enumerate(zip(y_pred, y_target.numpy().tolist())):
    if pred!=target:
      #print(i, pred, target)
      missclassified.append(i)
  print("total missclassified: ",len(missclassified))
  return missclassified

In [78]:
# 150 pics of each class unmutable
print("test_dataset:")
test_ds = image_dataset_from_directory(
    test_folder,
      class_names=["Bedroom","Bathroom","Dinning","Livingroom","Kitchen"],
      seed=None,
      validation_split=None, 
      subset=None,
      image_size= image_size,
      batch_size= batch_size,
      color_mode='rgb',
      shuffle=False 
  )

missclassified_matrix=[]
for nn in models_dict:
  print(nn)
  for f in folders:
    print(f)

    # -- train dataset for each folder
    train_path = reg_input+f+"/"+'train_ds/'
    # print("\n train dataset:", "\n", train_path)
    train_ds = image_dataset_from_directory(
        train_path,
        class_names=["Bedroom","Bathroom","Dinning","Livingroom","Kitchen"],
        seed=None,
        validation_split=None, 
        subset=None,
        image_size= image_size,
        batch_size= batch_size,
        color_mode='rgb',
        shuffle=False 
        )
    
    # inferences and real values
    y_pred, y_target = inferences_target_list(models_dict[nn], train_ds)
    
    # missclassified ones
    missclassified = get_missclassified(y_pred, y_target)
    missclassified_matrix.append(missclassified)

    # val_path = reg_input+f+"/"+"val_ds"
    # print("\n val dataset:", "\n", val_path)
    # val_ds = image_dataset_from_directory(
    #   val_path,
    #   class_names=["Bedroom","Bathroom","Dinning","Livingroom","Kitchen"],
    #   seed=None,
    #   validation_split=None, 
    #   subset=None,
    #   image_size= image_size,
    #   batch_size= batch_size,
    #   color_mode='rgb',
    #   shuffle=False 
    # )

    # class_names = train_ds.class_names
    # print(class_names)
print(missclassified_matrix)

test_dataset:
Found 750 files belonging to 5 classes.
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold0.h5
fold2
Found 3598 files belonging to 5 classes.
3598 3598
714
fold4
Found 3598 files belonging to 5 classes.
3598 3598
695
fold0
Found 3598 files belonging to 5 classes.
3598 3598
593
fold3
Found 3598 files belonging to 5 classes.
3598 3598
710
fold1
Found 3598 files belonging to 5 classes.
3598 3598
694
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold1.h5
fold2
Found 3598 files belonging to 5 classes.
3598 3598
694
fold4
Found 3598 files belonging to 5 classes.
3598 3598
692
fold0
Found 3598 files belonging to 5 classes.
3598 3598
685
fold3
Found 3598 files belonging to 5 classes.
3598 3598
697
fold1
Found 3598 files belonging to 5 classes.
3598 3598
579
resnet50_NOdataAug_dropoutFirst007_regKfolds_fold2.h5
fold2
Found 3598 files belonging to 5 classes.
3598 3598
384
fold4
Found 3598 files belonging to 5 classes.
3598 3598
469
fold0
Found 3598 files belonging to 5 classes.
3598 

In [83]:
# 5 lists for each model
len(missclassified_matrix)

25

In [90]:
final = [missclassified_matrix[0],
         missclassified_matrix[1],
         missclassified_matrix[2],
         missclassified_matrix[3],
         missclassified_matrix[4]]

In [91]:
elements_in_all = list(set.intersection(*map(set, list(
    final
))))
elements_in_all
i need to see if it's the same model in different folders or different models in the same folder (that's what i want) => check loop, automate this stuff 

[1314, 1988, 1925, 2022, 2026, 2702, 2703, 1902, 1873, 1909, 1242, 1499, 1980]

In [92]:
final = [missclassified_matrix[5],
         missclassified_matrix[6],
         missclassified_matrix[7],
         missclassified_matrix[8],
         missclassified_matrix[9]]
elements_in_all = list(set.intersection(*map(set, list(
    final
))))
elements_in_all

[1925, 2022, 2702, 2033, 1913]

In [18]:
finales de la semana que viene - 24-25

PARA CADA MODELO VER DÓNDE FALLA EN TRAIN, TEST, val
- QUÉ IMÁGENES SON LAS MÁS REPETIDAS Y FALLAN

- fold de 5. recorrer todos los val de todos los folders, para cada uno de los 5 modelos, en qué imágenes del test falla cada uno de ellos. 
- luego los 5 modelos con los mismos test

- para cada modelo en qué imágenes falla de los correspondientes conjuntos de validación => habrá imágenes para que todos los modelso no funcionen bien

QUE LA ETIQUETA NO SE CORRESPONDA CON LA PREDICCIÓN (olvida los numpy)

3 semanas para generación de imágenes y reentrenar modelos

1 par de semanas para escribir



SyntaxError: ignored